# Intelligent App with Google Generative AI and Neo4j

In [60]:
import os
import sys
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import AzureChatOpenAI
import re 
import os
from langchain_community.graphs import Neo4jGraph
import dotenv
import pandas as pd
import requests
sys.path.append(os.path.abspath(".."))
from helpers.df_helpers import documents2Dataframe
from helpers.clean_data import clean_text
import gcloud



In [97]:

project_id= "elevated-module-440602-p8"
location='us-central1'

In [62]:
import vertexai
from langchain_community.graphs import Neo4jGraph
import dotenv
import os
dotenv.load_dotenv()

vertexai.init(project=project_id,location=location)
graph=Neo4jGraph(url=os.getenv('NEO4J_URI_ONLINE'),
                 username=os.getenv('NEO4J_USERNAME_ONLINE'),
                 password=os.getenv('NEO4J_PASSWORD_ONLINE')
                #  database=os.getenv('NEO4J_DATABASE')
)

llm=AzureChatOpenAI(
    azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT_MODEL'),
    api_version=os.getenv('AZURE_OpenAI_API_VERSION'),
    temperature=0,
)
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001DE10ECCDF0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001DE0F6BFF10>, root_client=<openai.lib.azure.AzureOpenAI object at 0x000001DE0F6D94F0>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x000001DE10ECCE20>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), disabled_params={'parallel_tool_calls': None}, azure_endpoint='https://easttest123.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-03-15-preview', deployment_name='gpt-35-turbo', openai_api_version='2024-02-15-preview', openai_api_type='azure')

In [63]:
# prompt definition
import google.generativeai as palm
import os
palm.configure(api_key=os.getenv('GOOGLE_AI_API_KEY'))

model_list=[_ for _ in palm.list_models()]
model_list

[Model(name='models/chat-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 Chat (Legacy)',
       description='A legacy text-only model optimized for chat conversations',
       input_token_limit=4096,
       output_token_limit=1024,
       supported_generation_methods=['generateMessage', 'countMessageTokens'],
       temperature=0.25,
       max_temperature=None,
       top_p=0.95,
       top_k=40),
 Model(name='models/text-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 (Legacy)',
       description='A legacy model that understands text and generates text as an output',
       input_token_limit=8196,
       output_token_limit=1024,
       supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
       temperature=0.7,
       max_temperature=None,
       top_p=0.95,
       top_k=40),
 Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
      

In [64]:
url_input=("https://en.wikipedia.org/wiki/Satoshi_Nakamoto")
loader=WebBaseLoader([url_input])
data=loader.load().pop().page_content
from helpers.clean_data import clean_text
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

data=clean_text(data)
documents=[Document(page_content=data)]

# split the data into chunks 
splitter=RecursiveCharacterTextSplitter(chunk_size=3000,chunk_overlap=300)
smaller_doc=splitter.split_documents(documents)
smaller_doc[:4]

[Document(metadata={}, page_content='Satoshi Nakamoto WikipediaJump to contentMain menuMain menumove to sidebarhideNavigationMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usContributeHelpLearn to editCommunity portalRecent changesUpload fileSearchSearchAppearanceDonateCreate accountLog inPersonal toolsDonate Create account Log inPages for logged out editors learn moreContributionsTalkContentsmove to sidebarhideTop1Development of bitcoin2Characteristics and identityToggle Characteristics and identity subsection21Possible identities211Hal Finney212Dorian Nakamoto213Nick Szabo214Craig Wright215Other candidates3In popular culture4References5External linksToggle the table of contentsSatoshi Nakamoto52 languagesAfrikaansAzrbaycanca BnlmgCataletinaDanskDeutschEestiEspaolEsperantoFranaisHausaBahasa IndonesiaItalianoKiswahiliKurdLatvieuLombardMagyar NederlandsNorsk bokmlOccitanOzbekcha PolskiPortugusRomnShqipSimple EnglishSloveninaSuomiTagalogTrke UyghurcheTing VitEdit lin

In [ ]:
dynamic_entity_prompt_tpl="""From the biography text of a famous person or celebrity below, extract Entities and Relationships strictly as instructed:
1. Identify all significant entities and relationships from the text, categorizing each as described below. The `id` property of each entity must be unique and alphanumeric, to ensure distinct nodes in Neo4j. You will be using this `id` property to define relationships between entities. Only use the entity types below; do NOT create new types.

   **Entity Types:**
   - **Person**: label:'Person', id:string, role:string, description:string // Summary of the individual’s primary role or identity.
   - **Achievement**: label:'Achievement', id:string, description:string // Major accomplishments, awards, notable works, or contributions.
   - **Organization**: label:'Organization', id:string, role:string, description:string // Companies, institutions, or groups the person is associated with.
   - **Event**: label:'Event', id:string, description:string // Important events related to the individual (e.g., awards, major milestones).
   - **Relationship**: label:'Relationship', id:string, type:string, source:string, target:string // Defines the connection type (e.g., "Collaborated with," "Founded," "Won") and references `source` and `target` entities by `id`.

2. **Description Property Requirements**:
   - Each description should be a concise summary, no more than 100 characters, focused on the essential detail.
   - **Person**: Summarize their primary identity (e.g., "Renowned American actor").
   - **Achievement**: Specify the nature of the accomplishment (e.g., "Oscar-winning performance in 'Good Will Hunting'").
   - **Organization**: Indicate the person’s role or connection (e.g., "Co-founder of Tesla, Inc.").
   - **Event**: Highlight the key details of the event (e.g., "Nobel Prize in Physics, 1921").
   - **Relationship**: Clearly describe the connection and indicate its direction, ensuring each relationship is accurately categorized.

3. **Rules for Extraction**:
   - Avoid fictional or inferred data; extract only verifiable entities and relationships.
   - Do not create duplicate entities.
   - Only extract entities directly relevant to the person’s role, achievements, or associations.
   - Avoid personal anecdotes or unrelated information.

4. **Example Output JSON**:
{
  "entities": [
    {"label":"Person","id":"person1","role":"Actor","description":"Renowned American film actor"},
    {"label":"Achievement","id":"achievement1","description":"Oscar-winning performance in 'Good Will Hunting'"},
    {"label":"Organization","id":"organization1","role":"Co-founder","description":"Co-founder of Tesla, Inc."},
    {"label":"Event","id":"event1","description":"Nobel Prize in Physics, 1921"},
    {"label":"Relationship","id":"relationship1","type":"Collaborated with","source":"person1","target":"person2"}
  ]
}

Question: Now, extract the entities and relationships for the text below -
$ctext

Answer:
"""


In [66]:
from vertexai.preview.language_models import TextGenerationModel

def run_text_model(
    project_id:str,
    model_name:str,
    temperature:float,
    max_decode_steps:int,
    top_k: int,
    top_p: float,
    prompt: str,
    location: str=location,
    tuned_model_name:str=" ",
    ) :
    vertexai.init(project=project_id,location=location)
    model=TextGenerationModel.from_pretrained(model_name)
    if tuned_model_name:
        model=TextGenerationModel.from_pretrained(model_name)
    response=model.predict(
        prompt,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,
    )
    return response.text

In [100]:
def extract_entries_relationships(prompt,tune_model_name):
    try:
        res=run_text_model(project_id,'text-bison@001',0,1024,0.8,40,prompt,location,tune_model_name)
        return res
    except Exception as e:
        print(e)


In [101]:
data=""" 
Elon Reeve Musk FRS (/ˈiːlɒn/; born June 28, 1971) is a businessman and investor known for his key roles in the space company SpaceX and the automotive company Tesla, Inc. Other involvements include ownership of X Corp., the company that operates the social media platform X (formerly known as Twitter), and his role in the founding of the Boring Company, xAI, Neuralink, and OpenAI. He is one of the wealthiest individuals in the world; as of August 2024 Forbes estimates his net worth to be US$247 billion.[3]

Musk was born in Pretoria, South Africa, to Maye (née Haldeman), a model, and Errol Musk, a businessman and engineer. Musk briefly attended the University of Pretoria before immigrating to Canada at the age of 18, acquiring citizenship through his Canadian-born mother. Two years later he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but dropped out after two days and, with his brother Kimbal, co-founded the online city guide software company Zip2. The startup was acquired by Compaq for $307 million in 1999. That same year Musk co-founded X.com, a direct bank. X.com merged with Confinity in 2000 to form PayPal. In 2002 Musk acquired US citizenship. That October eBay acquired PayPal for $1.5 billion. Using $100 million of the money he made from the sale of PayPal, Musk founded SpaceX, a spaceflight services company, in 2002.

In 2004, Musk was an early investor who provided most of the initial financing in the electric-vehicle manufacturer Tesla Motors, Inc. (later Tesla, Inc.), assuming the position of the company's chairman. He later became the product architect and, in 2008, the CEO. In 2006, Musk helped create SolarCity, a solar energy company that was acquired by Tesla in 2016 and became Tesla Energy. In 2013, he proposed a hyperloop high-speed vactrain transportation system. In 2015, he co-founded OpenAI, a nonprofit artificial intelligence research company. The following year Musk co-founded Neuralink, a neurotechnology company developing brain–computer interfaces, and The Boring Company, a tunnel construction company. In 2018 the U.S. Securities and Exchange Commission (SEC) sued Musk, alleging that he had falsely announced that he had secured funding for a private takeover of Tesla. To settle the case Musk stepped down as the chairman of Tesla and paid a $20 million fine. In 2022, he acquired Twitter for $44 billion, merged the company into the newly-created X Corp. and rebranded the service as X the following year. In March 2023, Musk founded xAI, an artificial-intelligence company.

Musk's actions and expressed views have made him a polarizing figure.[4] He has been criticized for making unscientific and misleading statements, including COVID-19 misinformation, promoting right-wing conspiracy theories, and "endorsing an antisemitic theory"; he later apologized for the last of these.[5][4][6] His ownership of Twitter has been controversial because of the layoffs of large numbers of employees, an increase in hate speech, misinformation and disinformation posts on the website, and changes to website features, including verification. Musk has been active in American politics as a vocal and financial supporter of Donald Trump, becoming Trump's second-largest individual donor in October 2024.

"""

In [102]:
from string import Template
import json

results={'entities':[],'relationships':[]}
_prompt=Template(dynamic_entity_prompt_tpl).substitute(ctext=clean_text(data))
extraction=extract_entries_relationships(_prompt,'')
print(extraction)

400 Project `122385713024` is not allowed to use Publisher Model `projects/elevated-module-440602-p8/locations/us-central1/publishers/google/models/text-bison@001`
None


In [80]:
from google.auth.transport .requests import Request
from google.oauth2.service_account import Credentials

keypath='D:\pythonProjects\graphRAGAgent\service_account.json'

In [82]:
credentionals=Credentials.from_service_account_file(
    keypath
)

In [98]:
vertexai.init(project=project_id,location=location,credentials=credentionals)


In [103]:
from vertexai.language_models import TextGenerationModel
generation_model=TextGenerationModel.from_pretrained('text-bison@001')

In [104]:
generation_model.predict('what is AI?').text

FailedPrecondition: 400 Project `122385713024` is not allowed to use Publisher Model `projects/elevated-module-440602-p8/locations/us-central1/publishers/google/models/text-bison@001`

In [107]:
from google.cloud import aiplatform

def list_models(project_id, location="us-central1"):
    aiplatform.init(project=project_id, location=location)
    models = aiplatform.Model.list()
    for model in models:
        print(model.display_name, model.resource_name)

list_models( "elevated-module-440602-p8")
